In [1]:
import pandas as pd
import csv
from pathlib import Path  

mor_fer_rate = 1.0072
df = pd.read_excel('../data/raw/VICpopu16-21.xlsx')

def categorise22(row):  
    if row[2016] * mor_fer_rate * mor_fer_rate < row[2018] and row[2019] * mor_fer_rate * mor_fer_rate > row[2021]:
        # have a high prob that this district is influenced by Covid
        return max(row[2021] * mor_fer_rate, 0)
    else:
        return max((row[2021] - row[2016]) / 5 + row[2021], 0)

def categorise23(row):  
    if row[2016] * mor_fer_rate * mor_fer_rate < row[2018] and row[2019] * mor_fer_rate * mor_fer_rate > row[2021]:
        return max(row[2020] * mor_fer_rate * mor_fer_rate, 0)
    else:
        return max((row[2021] - row[2016]) / 5 * 2 + row[2021], 0)

df[2022] = df.apply(categorise22, axis=1)
df[2023] = df.apply(categorise23, axis=1)

filepath = Path('../data/curated/predictpopu.csv')  
df[['SA2 code',2022,2023]].to_csv(filepath, index=False)  




In [43]:
import pandas as pd
import csv
from pathlib import Path  

dff = pd.read_excel('../data/raw/VICincome.xlsx')
def rate(row): 
    if not isinstance(row['2018-19'], int):
        # entry is np, no income in this district
        return 0
    else:
        if isinstance(row['2014-15'], int):
            # calculate growth rate over the past 5 years
            return (row['2018-19'] - row['2014-15']) / 5 / row['2018-19']
        else:
            # 2014 entry is np, so use 2016 instead
            return (row ['2018-19'] - row ['2016-17']) / 5 / row['2018-19']

def predict2019(row):
    if not isinstance(row['2018-19'], int):
        # entry is np
        return 0.0
    return max(float(row['2018-19']) * (1 + 1 * row['growthrate']),0)
def predict2020(row):
    return max(row['2019']* (1 + 2 * row['growthrate']),0)
def predict2021(row):
    return max(row['2020']* (1 + 0.3 * row['growthrate']),0)
def predict2022(row):
    return max(row['2021']* (1 + 2 * row['growthrate']),0)
def predict2023(row):
    return max(row['2022']* (1 + 2 * row['growthrate']),0)

dff['growthrate'] = dff.apply(rate, axis=1)
dff['2019'] = dff.apply(predict2019, axis=1)
dff['2020'] = dff.apply(predict2020, axis=1)
dff['2021'] = dff.apply(predict2021, axis=1)
dff['2022'] = dff.apply(predict2022, axis=1)
dff['2023'] = dff.apply(predict2023, axis=1)

filepath = Path('../data/curated/predictincome.csv')  
dff[['SA2','2019','2020','2021','2022','2023']].to_csv(filepath, index=False)  

